Обычные модели GNN не могут быть использованы для работы с гетерографами, т.к. атрибуты разных типов узлов/связей не могут нормально обрабатываться одними и теми же функциями. 

Интуитивно очевидный вариант: реализовать независимые процедуры рассылки сообщений для каждого типа связей.

PyG предоставляет три способа создания моделей для работы с гетерографами:
1. Автоматическое преобразование обычной модели к модели, работающий с гетерографами (`nn.to_hetero()`, `nn.to_hetero_with_bases()`)
2. Описаний функций для разных типов связей при помощи `nn.conv.HeteroConv`;
3. Использование готовых или создание новых гетерогенных операторов.

## Автоматическое преобразование

Пока что не работает :(

PyG дает возможность автоматически конвертировать любую PyG GNN модель в гетерогенную модель.

In [1]:
import torch_geometric.transforms as T
from torch_geometric.nn import SAGEConv, to_hetero
import torch

E:\anaconda3\envs\gcn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [66]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        # PyG позволяет использовать отложенную инициализацию
        # размерностей; это удобно для гетерографов, где размерности
        # для разных типов могут меняться
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        # forward описываем так же, как описывали бы
        # для обычной модели
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

In [56]:
# создадим небольшой синтетический датасет для предсказания классов узлов
num_classes = 6
dataset = FakeHeteroDataset(num_graphs=1,
                            num_node_types=2,
                            num_edge_types=3,
                            num_classes=num_classes,
                            task='node',
                            transform=T.ToUndirected())
data = dataset[0]
# будем предсказывать класс узлов v0
# для порядка разобьем все узлы на обучающее и тестовое множество
data['v0']['train_mask'] = torch.zeros(data['v0']['x'].shape[0]).bernoulli(0.8).bool()
data.metadata()

(['v0', 'v1'],
 [('v1', 'e0', 'v1'),
  ('v0', 'e0', 'v0'),
  ('v0', 'e0', 'v1'),
  ('v1', 'rev_e0', 'v0')])

In [67]:
model = GNN(hidden_channels=64, out_channels=num_classes)
model = to_hetero(model, data.metadata(), aggr='sum')

# инициализируем размерности
with torch.no_grad():
     model(data.x_dict, data.edge_index_dict)

# основной цикл обучения
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
model.train()
for epoch in range(50):
     out = model(data.x_dict, data.edge_index_dict)
     # out - словарь с ключами - типами узлов
     out_v0 = out['v0']
     mask = data['v0'].train_mask
     loss = F.cross_entropy(out_v0[mask], data['v0'].y[mask])
     loss.backward()
     optimizer.step()
     optimizer.zero_grad()
     if not epoch % 5:
          print(f'{epoch=:3d} {loss.item()=:.2f}')

epoch=  0 loss.item()=1.95
epoch=  5 loss.item()=1.48
epoch= 10 loss.item()=1.21
epoch= 15 loss.item()=0.94
epoch= 20 loss.item()=0.68
epoch= 25 loss.item()=0.44
epoch= 30 loss.item()=0.26
epoch= 35 loss.item()=0.15
epoch= 40 loss.item()=0.09
epoch= 45 loss.item()=0.05


## Обертка HeteroConv

`torch_geometric.nn.conv.HeteroConv` позволяет описывать процесс рассылки сообщений для гетерографов; отличие от автоматического конвертера `to_hetero()` состоит в том, что для разных типов можно использовать различные операторы свертки. 

In [3]:
import torch_geometric.transforms as T
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch_geometric.datasets import FakeHeteroDataset

In [15]:
num_classes = 6
dataset = FakeHeteroDataset(num_graphs=1,
                            num_node_types=2,
                            num_edge_types=3,
                            num_classes=num_classes,
                            task='node',
                            transform=T.ToUndirected())
data = dataset[0]
data['v0']['train_mask'] = torch.zeros(data['v0']['x'].shape[0]).bernoulli(0.8).bool()
data.metadata()

(['v0', 'v1'],
 [('v0', 'e0', 'v0'),
  ('v1', 'e0', 'v0'),
  ('v1', 'e0', 'v1'),
  ('v0', 'rev_e0', 'v1')])

In [49]:
class HeteroGNN(nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            # HeteroConv ждет на вход словарь; ключ - тип связи, значение - оператор свертки
            # Важно: если в HeteroConv один из типов узлов не будет указан в качестве destination
            # то после проведения гетеросвертки он исчезнет из словаря x_dict
            # и произойдет исключение KeyError
            conv = HeteroConv({
                ('v0', 'e0', 'v0'): GCNConv(-1, hidden_channels),
                ('v1', 'e0', 'v0'): SAGEConv((-1, -1), hidden_channels),
                ('v0', 'rev_e0', 'v1'): GATConv((-1, -1), hidden_channels),
            }, aggr='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        # тут параметры уже являются словарями
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            # нужно явно прописать нелинейности для всех типов узлов
            x_dict = {key: x.relu() for key, x in x_dict.items()}
        return self.lin(x_dict['v0'])

In [53]:
model = HeteroGNN(hidden_channels=64, 
                  out_channels=num_classes,
                  num_layers=2)
                  
with torch.no_grad():  # инициализируем размерности слоев
     out = model(data.x_dict, data.edge_index_dict)

optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
model.train()
for epoch in range(50):
     out = model(data.x_dict, data.edge_index_dict)
     mask = data['v0'].train_mask
     loss = F.cross_entropy(out[mask], data['v0'].y[mask])
     loss.backward()
     optimizer.step()
     optimizer.zero_grad()
     if not epoch % 5:
          print(f'{epoch=:3d} {loss.item()=:.2f}')

epoch=  0 loss.item()=1.80
epoch=  5 loss.item()=1.66
epoch= 10 loss.item()=1.43
epoch= 15 loss.item()=1.08
epoch= 20 loss.item()=0.70
epoch= 25 loss.item()=0.34
epoch= 30 loss.item()=0.12
epoch= 35 loss.item()=0.03
epoch= 40 loss.item()=0.01
epoch= 45 loss.item()=0.01


## Специализированные модели

Надо отметить, что в PyG их гораздо больше, чем в DGL.

In [3]:
import torch_geometric.transforms as T
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, HGTConv
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch_geometric.datasets import FakeHeteroDataset

In [22]:
num_classes = 6
dataset = FakeHeteroDataset(num_graphs=1,
                            num_node_types=2,
                            num_edge_types=3,
                            num_classes=num_classes,
                            task='node',
                            )
data = dataset[0]
data['v0']['train_mask'] = torch.zeros(data['v0']['x'].shape[0]).bernoulli(0.8).bool()
data.metadata()

(['v0', 'v1'], [('v1', 'e0', 'v0'), ('v0', 'e0', 'v0'), ('v1', 'e0', 'v1')])

In [23]:
data

HeteroData(
  v0={
    x=[1120, 74],
    y=[1120],
    train_mask=[1120]
  },
  v1={ x=[883, 49] },
  (v1, e0, v0)={ edge_index=[2, 8791] },
  (v0, e0, v0)={ edge_index=[2, 11157] },
  (v1, e0, v1)={ edge_index=[2, 8784] }
)

In [41]:
class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        # преобразования для каждого типа узлов 
        # чтобы получить одинаковую размерность
        for node_type in data.node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        # набор сверток 
        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            # если out_channels HGTConv не будет делиться нацело
            # на num_heads, то все сломается
            conv = HGTConv(hidden_channels, 
                           hidden_channels, 
                           data.metadata(),
                           num_heads, 
                           group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for node_type, x in x_dict.items():
            x_dict[node_type] = self.lin_dict[node_type](x).relu_()
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return self.lin(x_dict['v0'])

In [43]:
model = HGT(hidden_channels=64, 
            out_channels=num_classes,
            num_heads=1,
            num_layers=1)
                  
with torch.no_grad():  # инициализируем размерности слоев
     out = model(data.x_dict, data.edge_index_dict)

optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
model.train()
for epoch in range(50):
     out = model(data.x_dict, data.edge_index_dict)
     mask = data['v0'].train_mask
     loss = F.cross_entropy(out[mask], data['v0'].y[mask])
     loss.backward()
     optimizer.step()
     optimizer.zero_grad()
     if not epoch % 5:
          print(f'{epoch=:3d} {loss.item()=:.2f}')

epoch=  0 loss.item()=1.80
epoch=  5 loss.item()=1.74
epoch= 10 loss.item()=1.60
epoch= 15 loss.item()=1.45
epoch= 20 loss.item()=1.26
epoch= 25 loss.item()=1.04
epoch= 30 loss.item()=0.78
epoch= 35 loss.item()=0.54
epoch= 40 loss.item()=0.53
epoch= 45 loss.item()=0.24
